In [2]:
import numpy as np
import pandas as pd

In [5]:
file_name = "/content/sample_data/talks_dataset.df"

In [6]:
load_df = pd.read_pickle(file_name)
load_df.sample(3)

,speaker,query,companions,close_reply,neutral_reply,bad_reply,context
18680,Lisa Simpson,I thought you didn't like her.,"[Lisa Simpson, Homer Simpson, Marge Simpson, B...","[We're the neighbors, and we don't think., Wel...",[Hotel? Why don't you just stay at Ned's place...,"[Just with indifference., You should've though...","Are you blushing, Mr. Flanders?. Flanders has ..."
21530,Lisa Simpson,Why don't we go to our Congressman?,"[Lisa Simpson, Homer Simpson, Marge Simpson, H...",[I've been saying it for three weeks! But ever...,[That's a great idea. Why didn't you say somet...,"[Compliments of the deacon!, I'm so glad you w...","Now Cookie, do you think you can sell this hou..."
33408,Lisa Simpson,As big as a baby's head.,"[Lisa Simpson, Bart Simpson, Homer Simpson, Ma...","[That baby's head., Oh, man!, Yes! I'm so happ...","[Okay., Okay., Okay.]",[There's got to be a way to get that tree back...,"Bart, I am putting this money safely in the ba..."


In [7]:
!pip install trl transformers accelerate git+https://github.com/huggingface/peft.git -Uqqq
!pip install bitsandbytes einops wandb -Uqqq
#!pip install datasets -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [12]:
import torch
import glob
import pandas as pd
import numpy as np
import re
from peft import get_peft_model, PeftConfig, PeftModel, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from trl import SFTTrainer
from datasets import Dataset
import warnings
import random
warnings.filterwarnings("ignore")

In [8]:
!pip install accelerate

In [9]:
!pip install -i https://pypi.org/simple/ bitsandbytes

Looking in indexes: https://pypi.org/simple/


In [14]:
trained_model_dir = '/content/sample_data/tinyllama/checkpoint-700'

config = PeftConfig.from_pretrained(trained_model_dir)

In [15]:
# Loading the model with double quantization
model_name = "PY007/TinyLlama-1.1B-step-50K-105b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)


trained_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map='auto'

    )


config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [16]:
# Creating tokenizer and defining the pad token
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, padding_side='right')
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [19]:
# Generating lyrics with the base model. The repetition penalty in the generation config prevents the model from continually repeating the same string.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def generate_answer(query, model):
    encoding = tokenizer(query, return_tensors="pt").to(device)
    generation_config = GenerationConfig(max_new_tokens=250, pad_token_id = tokenizer.eos_token_id,repetition_penalty=1.3, eos_token_id = tokenizer.eos_token_id)
    outputs = model.generate(input_ids=encoding.input_ids, generation_config=generation_config)
    text_output = tokenizer.decode(outputs[0],skip_special_tokens=True)
    return text_output.split("[/INST]")[1].split("\n\n")[0]


In [20]:
import random

def get_test_prompt(df):
    # Определяем список колонок, которые будем использовать
    use_col = ['speaker', 'query', 'context', "companions", "close_reply"]

    # Генерируем случайный индекс строки из DataFrame
    j = random.randint(0, df.shape[0] - 1)

    # Генерируем случайный индекс для выбора элемента из "companions"
    companions_length = len(df.at[j, 'companions'])
    if companions_length == 0:
        id = 0  # Если список пуст, устанавливаем индекс в 0
    else:
        id = random.randint(0, companions_length - 1)

    # Извлекаем необходимые значения из DataFrame
    query = df.at[j, 'query']
    role = df.at[j, 'companions'][id]
    context = df.at[j, 'context']
    reply = df.at[j, 'close_reply'][id]

    # Формируем текст промптинга
    prompt = f"[INST]\n"
    prompt += "Use the given context to guide your answer about the query as indicated by your role:\n\n"
    prompt += f"Query: {query}\n\n"
    prompt += f"Context: {context}\n\n"
    prompt += f"Your role: {role}\n\n"
    prompt += "Answer:[/INST]"

    return query, role, reply, prompt

In [21]:
query, role, reply, prompt = get_test_prompt(load_df)
answer = generate_answer(prompt, trained_model)

print(f"\n\nQuery: {query}\n")
print(f"Trained_model reply with role: {role}\n")

print(answer)
print()
print(f'Answer in df: {reply}')



Query: You must hate me, Marge. Every time I try to help, I just fail miserably.

Trained_model reply with role: Homer Simpson



Answer in df: Hey, that's the fire alarm!
